<h1><center>New Shopping Mall in Kuala Lumpur, Malaysia</center></h1>


<h4><center>Applied Data Science Capstone</center></h4>
 

<h4><center>Coursera Capstone</center></h4>



<h3><center>By: Gyaneshwar Patle</center></h3>



<h4><center>July 2019</center></h4>

#  Capstone Course

**_New Shopping Mall in Kuala Lumpur, Malaysia_**

###  Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### Data Wikipedia page into a DataFrame

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text # send the GET request
soup = BeautifulSoup(data, 'html.parser') # parse data from the html into a beautifulsoup object
neighborhoodList = [] # create a list to store neighborhood data
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"): # append the data into the list
    neighborhoodList.append(row.text)

geographical_coordinates = pd.DataFrame({"Neighborhood": neighborhoodList}) # create a new DataFrame from the list
print(geographical_coordinates.shape) # print the number of rows of the dataframe
geographical_coordinates.head(20)

(71, 1)


,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan
5,Bandar Tun Razak
6,Bangsar
7,Bangsar Park
8,Bangsar South
9,Batu 11 Cheras


### Get the geographical coordinates

In [3]:
def get_latlng(neighborhood):   # define a function to get coordinates
    lat_lng_coords = None # initialize your variable to None
    while(lat_lng_coords is None):     # loop until you get the coordinates
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# call the function to get the coordinates, store in a new list using list comprehension
coordinates = [ get_latlng(neighborhood) for neighborhood in geographical_coordinates["Neighborhood"].tolist() ]

In [4]:
coordinates

[[3.0576900000000364, 101.74388000000005],
 [3.1531525006886456, 101.70041313127312],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072620000000029, 101.71471000000008],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1347800000000348, 101.67262000000005],
 [3.111020000000053, 101.66283000000004],
 [3.09898000000004, 101.73499000000004],
 [3.1357600000000616, 101.70837000000006],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.057800000000043, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.152017197420035, 101.70102760046613],
 [3.129290000000026, 101.69892000000004],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.147980000000075, 101.66798000000006],
 [3.1387586696676304, 101.6840455304707],
 [3.1387586696676304, 101.6840455304707],
 [3.156685250705136, 101.69807

In [5]:
df_coords = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude']) # create temporary dataframe to populate the coordinates into Latitude and Longitude
geographical_coordinates['Latitude'] = df_coords['Latitude'] # merge the coordinates into the original dataframe
geographical_coordinates['Longitude'] = df_coords['Longitude']
print(geographical_coordinates.shape) # check the neighborhoods and the coordinates
geographical_coordinates.head()

(71, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.153153,101.700413
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072620,101.714710


### save the geographical coordinates as CSV file

In [6]:
geographical_coordinates.to_csv("geographical_coordinates.csv", index=False)

### Create a map of Kuala Lumpur with neighborhoods superimposed on top

In [7]:

# Remove "#" from below line and change 'user_agent' that one is created on 'Foursquare'

# get the coordinates of Kuala Lumpur
address = 'Kuala Lumpur, Malaysia'


geolocator = Nominatim(user_agent="Coursera_Captone_Project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516636, 101.6943028.


### create map of Toronto using latitude and longitude values

In [8]:

map_created = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(geographical_coordinates['Latitude'], geographical_coordinates['Longitude'], geographical_coordinates['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_created)  
    
map_created

### save the map as HTML file

In [9]:
map_created.save('map_created.html')

### Foursquare API to explore the neighborhoods

In [10]:
# define Foursquare Credentials and Version
CLIENT_ID = 'TB5WVST1CIZ0VHX52C0COSTQGDOADSUOBAOQF3PZCXI31BIT' # your Foursquare ID
CLIENT_SECRET = '2CW0FUOCFOBMKIP2AZ4YH1BILDJVIRQIIEDVAP3PSJTJHLGJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TB5WVST1CIZ0VHX52C0COSTQGDOADSUOBAOQF3PZCXI31BIT
CLIENT_SECRET:2CW0FUOCFOBMKIP2AZ4YH1BILDJVIRQIIEDVAP3PSJTJHLGJ


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [11]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(geographical_coordinates['Latitude'], geographical_coordinates['Longitude'], geographical_coordinates['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [12]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7088, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
2,Alam Damai,3.05769,101.74388,Restoran Ikbal,3.061134,101.750220,Restaurant
3,Alam Damai,3.05769,101.74388,閒茶素食店 Leisure Tea Vegetarian,3.057673,101.747258,Vegetarian / Vegan Restaurant
4,Alam Damai,3.05769,101.74388,Ivy Sekinchan Seafood Noodle House 适耕莊特制魚丸海鲜面,3.065749,101.748718,Noodle House


**Let's check how many venues were returned for each neighorhood**

In [13]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,99,99,99,99,99,99
Bandar Tun Razak,100,100,100,100,100,100
Bangsar,100,100,100,100,100,100
Bangsar Park,100,100,100,100,100,100
Bangsar South,100,100,100,100,100,100


**Let's find out how many unique categories can be curated from all the returned venues**

In [14]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 300 uniques categories.


In [15]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Supplement Shop', 'Noodle House', 'Restaurant',
       'Vegetarian / Vegan Restaurant', 'Chinese Restaurant',
       'Food Court', 'Breakfast Spot', 'Other Great Outdoors',
       'Asian Restaurant', 'Park', 'Dim Sum Restaurant', 'Snack Place',
       'Food Truck', 'Japanese Restaurant', 'Indian Restaurant',
       'Bubble Tea Shop', 'Spa', 'Seafood Restaurant',
       'Chinese Breakfast Place', 'Convenience Store', 'Dessert Shop',
       'Cantonese Restaurant', 'Bakery', 'Café', 'Farmers Market',
       'Malay Restaurant', 'Outlet Store', 'Hakka Restaurant',
       'Gym / Fitness Center', 'Pet Store', 'Steakhouse',
       'Badminton Court', 'Athletics & Sports', 'Fast Food Restaurant',
       'Middle Eastern Restaurant', 'Burger Joint',
       'Padangnese Restaurant', 'Winery', 'Grocery Store',
       'College Bookstore', 'Monument / Landmark', 'Hostel', 'Hotel',
       'Latin American Restaurant', 'Hotel Pool',
       'South Indian Restaurant', 'Soup Place', 'Dance Studio',
 

In [16]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [17]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(7088, 301)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Airport Terminal,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Padangnese Restaurant,Pakistani Restaurant,Palace,Park,Parking,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tunnel,T

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [18]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(71, 301)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Airport Terminal,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Padangnese Restaurant,Pakistani Restaurant,Palace,Park,Parking,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tunnel,T

In [19]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

40

**Create a new DataFrame for Shopping Mall data only**

In [20]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [21]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Alam Damai,0.000000
1,"Ampang, Kuala Lumpur",0.030000
2,Bandar Menjalara,0.000000
3,Bandar Sri Permaisuri,0.000000
4,Bandar Tasik Selatan,0.010101


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [22]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 2, 2, 0, 2, 1, 1, 1, 2])

In [23]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [24]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.000000,2
1,"Ampang, Kuala Lumpur",0.030000,1
2,Bandar Menjalara,0.000000,2
3,Bandar Sri Permaisuri,0.000000,2
4,Bandar Tasik Selatan,0.010101,0


In [25]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(geographical_coordinates.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.000000,2,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.030000,1,3.153153,101.700413
2,Bandar Menjalara,0.000000,2,3.190350,101.625450
3,Bandar Sri Permaisuri,0.000000,2,3.103910,101.712260
4,Bandar Tasik Selatan,0.010101,0,3.072620,101.714710


In [26]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
70,Wangsa Maju,0.010000,0,3.203910,101.737190
21,Damansara Town Centre,0.010000,0,3.138759,101.684046
22,"Damansara, Kuala Lumpur",0.010000,0,3.138759,101.684046
23,Dang Wangi,0.020000,0,3.156685,101.698076
50,Shamelin,0.020000,0,3.124580,101.735970
27,"Jalan Cochrane, Kuala Lumpur",0.020000,0,3.132977,101.724669
28,Jalan Duta,0.010000,0,3.180163,101.677880
32,Kampung Datuk Keramat,0.010000,0,3.166400,101.730460
20,Damansara Heights,0.020000,0,3.147980,101.667980
69,Titiwangsa,0.010000,0,3.180670,101.703220


**Finally, let's visualize the resulting clusters**

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [29]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
70,Wangsa Maju,0.010000,0,3.203910,101.737190
21,Damansara Town Centre,0.010000,0,3.138759,101.684046
22,"Damansara, Kuala Lumpur",0.010000,0,3.138759,101.684046
23,Dang Wangi,0.020000,0,3.156685,101.698076
50,Shamelin,0.020000,0,3.124580,101.735970
27,"Jalan Cochrane, Kuala Lumpur",0.020000,0,3.132977,101.724669
28,Jalan Duta,0.010000,0,3.180163,101.677880
32,Kampung Datuk Keramat,0.010000,0,3.166400,101.730460
20,Damansara Heights,0.020000,0,3.147980,101.667980
69,Titiwangsa,0.010000,0,3.180670,101.703220


#### Cluster 1

In [30]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
66,Taman Tun Dr Ismail,0.03,1,3.152830,101.622710
6,Bangsar,0.05,1,3.129200,101.678440
67,Taman U-Thant,0.04,1,3.157700,101.724520
8,Bangsar South,0.03,1,3.111020,101.662830
36,Lembah Pantai,0.05,1,3.121202,101.663899
42,"Pudu, Kuala Lumpur",0.04,1,3.133540,101.713070
7,Bangsar Park,0.05,1,3.134780,101.672620
15,Bukit Nanas,0.03,1,3.152017,101.701028
31,"Kampung Baru, Kuala Lumpur",0.03,1,3.165460,101.710280
30,KL Eco City,0.04,1,3.117140,101.673890


#### Cluster 2

In [31]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
54,Taman Bukit Maluri,0.0,2,3.200660,101.633370
55,Taman Cheras Hartamas,0.0,2,3.082630,101.746710
52,Sri Petaling,0.0,2,3.072600,101.682520
68,Taman Wahyu,0.0,2,3.222400,101.671730
61,Taman Midah,0.0,2,3.093590,101.728370
62,Taman OUG,0.0,2,3.210050,101.634508
63,Taman P. Ramlee,0.0,2,3.193600,101.705980
58,Taman Ibukota,0.0,2,3.212160,101.715400
59,Taman Len Seng,0.0,2,3.069080,101.742870
60,Taman Melati,0.0,2,3.223570,101.723990
